<a href="https://colab.research.google.com/github/MaximeFagnan/polyenum2025/blob/main/polyEnum_Jensen/jupyterNB_polyEnum_Jensen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jensen's algorithm simplified

Jensen's algorithm for polyomino enumeration

### data structures

In [52]:
from typing import Tuple

class Signature:

    #This initializer makes a signature from a set of states, it supposes that
        # these states are correct
    def __init__(self, states, touches_top = False, touches_bottom = False):
        self.length = len(states)
        self.states : tuple[int,...] = tuple(states)
        self.touches_top : bool = touches_top
        self.touches_bottom: bool = touches_bottom
        self._hash = self.compute_hash()

    # # Turns out we never have to clone a signature (it should be immutable anyways)
    # def clone(self):
    #     return Signature(self.states, self.touches_top, self.touches_bottom)

    def __eq__(self, other) -> bool:
        if not isinstance(other, Signature):
            return False
        if self._hash != other._hash:
            return False
        return (self.states == other.states and
                self.touches_top == other.touches_top and
                self.touches_bottom == other.touches_bottom)

    def compute_hash(self):
        base_hash = hash(self.states)
        flags_hash = (self.touches_top << 1) | self.touches_bottom
        return hash((base_hash, flags_hash))

    def __hash__(self) -> int:
        return self._hash

    def __repr__(self, multiline : bool = True) -> str:
        repr_list = []

        touch_top_symbol = ""
        if self.touches_top:
            if multiline:
                touch_top_symbol = "↧"
            else:
                touch_top_symbol = "↦"
        touch_bottom_symbol = ""
        if self.touches_bottom:
            if multiline:
                touch_bottom_symbol = "↥"
            else:
                touch_bottom_symbol = "↤"

        # states[0] + touches_top flag
        if multiline:
            repr_list.append(f"{self.states[0]}{touch_top_symbol}")
        else:
            repr_list.append(f"{touch_top_symbol}{self.states[0]}")
        # states[1:-1] on one line
        repr_list += list(map(str, self.states[1:-1]))
        # states[-1] + touches_bottom flag
        if self.length > 1:
            repr_list.append(f"{self.states[-1]}{touch_bottom_symbol}")

        if multiline:
            return "\n".join(repr_list)
        else:
            return " ".join(repr_list)

    def can_add(self, row):
        """ Check if this signature's transition is a valid "add" transition
        when setting row to 0."""
        # quick check, not likely to be true
        if self.states[row] != 1:
            return False
        for row_index, state in enumerate(self.states):
            if row_index == row:
                pass # already verified
            else:
                if state != 0:
                    return False
        # The signature needs to touch the bottom and the top
        return self.touches_top and self.touches_bottom

    def mirrored(self):
        # should only be called on odd length signatures
        assert self.length % 2 == 1, "Should only mirror odd length signatures"
        # Swap 2 ↔ 4 to preserve matching semantics when vertically flipped
        swap = {0: 0, 1: 1, 2: 4, 3: 3, 4: 2}
        mirrored_states = [swap[state] for state in reversed(self.states)]
        return Signature(mirrored_states, self.touches_bottom, self.touches_top)

    def calculate_n_add(self, column, row_after_kink):
        #column refers to which column is being worked on
        assert 0 <= row_after_kink < self.length

        # first calculate n_w, where n_w is number of cells to reach width=length
        n_w = 0
        for row, state in enumerate(self.states[:row_after_kink+1]):
            if row < row_after_kink:
                if state > 0:
                    n_w = self.length - column
                    break
            else:
                n_w = self.length - column + 1
                break #unnecessary because we truncated states

        # now calculate n_h which is necessary number of polyominoes to reach top and bottom
        n_h = 0
        if not self.touches_top:
            # find highest occupied cell
            for row, state in enumerate(self.states):
                if state > 0:
                    n_h += row
                    break
        if not self.touches_bottom:
            # find lowest occupied cell
            for row, state in enumerate(reversed(self.states)):
                if state > 0:
                    n_h += row
                    break

        # now calculate n_c which is the number of cells required to connect the different components
        n_c = 0
        # we could choose to have n_c be too small if it makes it simpler.






# keys: (modify_to, lower, upper)
# values: (operation, new_lower, new_upper)
TRANSITION_TABLE = {
    # modify_to = 0 (unoccupied cell)
    (0, 0, 0): ("blank", 0, 0),
    (0, 0, 1): ("blank", 0, 1),
    (0, 0, 2): ("blank", 0, 2),
    (0, 0, 3): ("blank", 0, 3),
    (0, 0, 4): ("blank", 0, 4),

    (0, 1, 0): ("impossible", 0, 0), #add operation verified in control loop, otherwise it's invalid
    (0, 1, 1): ("impossible", 0, 0),
    (0, 1, 2): ("impossible", 0, 0),
    (0, 1, 3): ("impossible", 0, 0),
    # (0, 1, 4): ("impossible", 0, 0), # should never reach this state

    (0, 2, 0): ("over-line", 0, 0),
    (0, 2, 1): ("over-line", 0, 1),
    (0, 2, 2): ("over-line", 0, 2),
    (0, 2, 3): ("blank", 0, 2),
    (0, 2, 4): ("blank", 0, 1),

    (0, 3, 0): ("blank", 0, 0),
    (0, 3, 1): ("blank", 0, 1),
    (0, 3, 2): ("blank", 0, 2),
    (0, 3, 3): ("blank", 0, 3),
    (0, 3, 4): ("blank", 0, 4),

    (0, 4, 0): ("over-line", 0, 0),
    (0, 4, 1): ("over-line", 0, 1),
    (0, 4, 2): ("over-line", 0, 2),
    (0, 4, 3): ("over-line", 0, 3),
    # (0, 4, 4): ("impossible", 0, 0), # should never reach this state

    # modify_to = 1 (occupied cell)
    (1, 0, 0): ("blank", 1, 0),
    (1, 0, 1): ("blank", 2, 4),
    (1, 0, 2): ("blank", 2, 3),
    (1, 0, 3): ("blank", 3, 3),
    (1, 0, 4): ("blank", 3, 4),

    (1, 1, 0): ("blank", 1, 0),
    (1, 1, 1): ("blank", 2, 4),
    (1, 1, 2): ("blank", 2, 3),
    (1, 1, 3): ("blank", 3, 3),
    # (1, 1, 4): ("impossible", 0, 0), # should never reach this state

    (1, 2, 0): ("blank", 2, 0),
    (1, 2, 1): ("blank", 2, 3),
    (1, 2, 2): ("hat", 2, 3),
    (1, 2, 3): ("blank", 2, 3),
    (1, 2, 4): ("blank", 2, 4),

    (1, 3, 0): ("blank", 3, 0),
    (1, 3, 1): ("blank", 3, 3),
    (1, 3, 2): ("hat", 3, 3),
    (1, 3, 3): ("blank", 3, 3),
    (1, 3, 4): ("blank", 3, 4),

    (1, 4, 0): ("blank", 4, 0),
    (1, 4, 1): ("blank", 3, 4),
    (1, 4, 2): ("blank", 3, 3),
    (1, 4, 3): ("hat", 3, 3),
    # (1, 4, 4): ("impossible", 0, 0) # should never reach this state
}


class GeneratingFunction:
    def __init__(self, terms=None, max_degree = None, min_exp = None):
        self.terms = terms or {}  # {exponent: coefficient}
        self.max_degree = max_degree
        if min_exp is not None:
            self.min_exp = min_exp
        else:
            self.min_exp = min(self.terms) if self.terms else 0


    def __add__(self, other):
        assert self.max_degree == other.max_degree, "Mismatched degree bounds"
        result_terms = self.terms.copy()
        min_exp = min(self.min_exp, other.min_exp)
        for exp, coeff in other.terms.items():
            result_terms[exp] = result_terms.get(exp, 0) + coeff
        return GeneratingFunction(result_terms, self.max_degree, min_exp)

    def multiply_by_x(self):
        self.terms = {exp + 1: coeff for exp, coeff in self.terms.items()}
        # Delete last term if it's power is greater than max_degree
        if self.max_degree is not None and self.max_degree in self.terms:
            del self.terms[self.max_degree]
        self.min_exp += 1

    def __repr__(self):
        return " + ".join(f"{coeff}x^{exp}" for exp, coeff in sorted(self.terms.items()))

    def clone(self):
        return GeneratingFunction(self.terms.copy(), self.max_degree, self.min_exp)

    # # We do not need to truncate because we dynamically manage max degree.
    # def truncate(self, max_n):
    #     """
    #     Remove all terms with exponent ≥ max_n (typically to keep area < n).
    #     This mutates the generating function in place.
    #     """
    #     self.terms = {exp: coeff for exp, coeff in self.terms.items() if exp < max_n}



class SignatureGFPair:
    def __init__(self, signature, generating_function):
        self.signature = signature
        self.generating_function = generating_function

    def __eq__(self, other):
        return self.signature == other.signature

    def __hash__(self):
        return hash(self.signature)

    def __repr__(self):
        return f"{self.signature} -> {self.generating_function}"

    def __add__(self, other):
        assert self.signature == other.signature, "Signatures must match for addition"
        new_generating_function = self.generating_function + other.generating_function
        return SignatureGFPair(self.signature, new_generating_function)

    def transition(self, row, modify_to):
        lower = self.signature.states[row]
        upper = self.signature.states[row-1] if row > 0 else 0
        key = (modify_to, lower, upper)

        if key not in TRANSITION_TABLE:
            return None  # skip invalid transitions

        operation, new_lower, new_upper = TRANSITION_TABLE[key]

        if operation == "impossible":
            return None  # no valid transition

        # Copy the old signature and modify it
        new_signature = None
        new_states = list(self.signature.states)
        touched_top = self.signature.touches_top
        touched_bottom = self.signature.touches_bottom
        if row == 0 and modify_to == 1:
            touched_top = True
        if row == len(new_states) - 1 and modify_to == 1:
            touched_bottom = True
        new_gf = self.generating_function.clone()
        if modify_to == 1:
            new_gf.multiply_by_x()

        # blank operation, just change states and nothing else
        if operation == "blank":
            new_states[row] = new_lower
            if row > 0:
                new_states[row-1] = new_upper

            new_signature = Signature(new_states, touched_top, touched_bottom)

        # over-line operation
        elif operation == "over-line":
            new_states[row] = new_lower  # which is 0

            if lower == 2:
                count_2 = 1
                count_4 = 0
                for i in range(row-1, -1, -1):
                    if new_states[i] == 2:
                        count_2 += 1
                    if new_states[i] == 4:
                        count_4 += 1
                    """If we find a '3' and the current nesting counts match
                    (minus one for the removed 2), then we can turn it into 2"""
                    if new_states[i] == 3  and (count_2-1 == count_4):
                        new_states[i] = 2 # it becomes the new 'first' site in the chain
                        break
                    """ If the count balances, then we just hit a balancing '4'
                    and there are only 2 cells connected to the signature"""
                    if count_2 == count_4:
                        new_states[i] = 1 # this is now an isolated occupied site
                        break

            elif lower == 4:
                count_4 = 1
                count_2 = 0
                for i in range(row+1, len(new_states)):
                    if new_states[i] == 2:
                        count_2 += 1
                    if new_states[i] == 4:
                        count_4 += 1
                    """If we find a '3' and the current nesting counts match
                    (minus one for the removed 2), then we can turn it into 2"""
                    if new_states[i] == 3  and (count_4-1 == count_2):
                        new_states[i] = 4 # it becomes the new 'first' site in the chain
                        break
                    """ If the count balances, then we just hit a balancing '2'
                    and there are only 2 cells connected to the signature"""
                    if count_2 == count_4:
                        new_states[i] = 1 # this is now an isolated occupied site
                        break
            new_signature = Signature(new_states, touched_top, touched_bottom)

        # hat operation
        elif operation == "hat":
            assert row > 0, "Should not be doing hat if row = 0"

            if new_states[row] in {2,3}:
                new_states[row] = new_lower
                new_states[row - 1] = 3  # upper site becomes intermediate

                # Scan upward from to find the matching 4
                count_2 = 0
                count_4 = 0
                for i in range(row-1,-1,-1):
                    if new_states[i] == 2:
                        count_2 += 1
                    elif new_states[i] == 4:
                        count_4 += 1
                        if (count_2 + 1) == count_4:
                            new_states[i] = 3  # last site also becomes intermediate
                            break

            if new_states[row] == 4 :
                new_states[row] = 3 # new site becomes intermediary

                # Scan downwards to find the matching 2
                count_2 = 0
                count_4 = 0
                for i in range(row+1,len(new_states)):
                    if new_states[i] == 2:
                        count_2 += 1
                        if count_2 == (count_4 + 1):
                            new_states[i] = 3  # first site also becomes intermediate
                            break
                    elif new_states[i] == 4:
                        count_4 += 1


            new_signature = Signature(new_states, touched_top, touched_bottom)

        # add operation
        # we turned add operation into impossible (but should be added in control loop)
        # assert operation != "add", "Add operation should not be reached"

        # Return new pair
        assert new_signature is not None, "New signature should not be None"
        return SignatureGFPair(new_signature, new_gf)


# SignatureTable
# ==============
#
# This class manages a collection of (Signature, GeneratingFunction) pairs during the execution
# of Iwan Jensen's polyomino enumeration algorithm.
#
# The table represents the current state of the dynamic programming process:
# it associates each unique boundary Signature with its corresponding GeneratingFunction.
# This class is wrapper for the dictionary with an added symmetry merging optimization function
class SignatureTable:
    def __init__(self):
        self.mapping = {}

    def add(self, pair: SignatureGFPair):
        if pair.signature in self.mapping:
            self.mapping[pair.signature] += pair.generating_function
        else:
            self.mapping[pair.signature] = pair.generating_function

    # convenience function
    def items(self):
        return self.mapping.items()

    def __repr__(self, multiline = False):
        return "\n".join(f"{sig.__repr__(multiline = multiline)} -> {gf}" for sig, gf in self.mapping.items())

    def merge_mirrored_signatures(self):
        new_mapping = {}
        seen = set()

        for sig, gf in self.mapping.items():
            if sig in seen:
                continue

            mirrored_sig = sig.mirrored()
            seen.add(sig)
            seen.add(mirrored_sig)

            if mirrored_sig in self.mapping and mirrored_sig != sig:
                merged_gf = gf + self.mapping[mirrored_sig]
                new_mapping[sig] = merged_gf
            else:
                new_mapping[sig] = gf  # either symmetric or mirror not present

        self.mapping = new_mapping


In [53]:
# This is the polyomino enumerator class.
# initialize with a given n for which you want to enumerate and call the run
# method to get back the generating function of coefficients a_i up to i=n
class PolyominoEnumerator:
    def __init__(self, n):
        self.n = n
        self.w_max = (n + 1) // 2 # might be issue for odd n's ?
        self.total_gf = GeneratingFunction({0:1},n+1)
        # self.run()

    def run(self) -> GeneratingFunction:
        self.total_gf = GeneratingFunction({0:1},n+1) # reset gf
        for w in range(1,self.w_max+1):
            self.run_for_w(w)
        # print(self.total_gf)
        return self.total_gf

    def run_for_w(self, w) -> None:
        # signature table with only an empty signature of height w
        initial_signature = Signature([0]*w)
        initial_gf = GeneratingFunction({0:1},n+1)
        initial_pair = SignatureGFPair(initial_signature, initial_gf)
        table = SignatureTable()
        table.add(initial_pair)

        L_max = 2*self.w_max - w + 1
        # for every necessary column
        for col in range(1, L_max + 1):
            # Just make sure you flush out the all 0 signature after first row
            if col == 2:
                table.mapping.pop(initial_signature)
            # Pruning proposed by Gill Barequet and Gil Ben-Shachar (2024)
            if w % 2 == 1:
                table.merge_mirrored_signatures()
            # Move the kink down row by row
            for row in range(w):
                # make a new table
                new_table = SignatureTable()
                for sig, gf in table.items():

                    #first, add an occupied cell, update signature table
                    sig_gf = SignatureGFPair(sig, gf)
                    new_pair = sig_gf.transition(row, 1)
                    if new_pair is not None:
                        new_table.add(new_pair)

                    # Second, add an unoccupied cell
                    # Option 1: It seals a valid polyomino
                    if sig.can_add(row):
                        length = col-1
                        if length < w:
                            pass # ignore based on symmetry
                        elif length == w:
                            self.total_gf += gf
                        elif length > w:
                            self.total_gf += gf + gf # double based on symmetry

                    # Option 2: It does not create a valid polyomino, update signature table
                    else:
                        new_pair = sig_gf.transition(row, 0)
                        if new_pair is not None:
                            new_table.add(new_pair)

                # Update the signatureGF table after every move of the kink
                table = new_table
        return # Does not return anything since the total_gf is a parameter of the class directly

In [54]:
# Testing cell
def test_signature_class():
    states = [4,0,4,3,2,0,2,0,1]
    signature_test1 = Signature(states)
    signature_test2 = Signature(states, True, True)

    print(signature_test1.__repr__(multiline = False))
# test_signature_class()

def test_signature_table():
    # Define some signatures
    sig1 = Signature([4, 3, 2], touches_top=True)
    sig2 = Signature([4, 3, 2], touches_top=True)  # equal to sig1
    sig3 = Signature([0, 1, 0], touches_top=True)

    # Define generating functions
    gf1 = GeneratingFunction({1: 2})
    gf2 = GeneratingFunction({2: 3})
    gf3 = GeneratingFunction({0: 5})

    # Create pairs
    pair1 = SignatureGFPair(sig1, gf1)
    pair2 = SignatureGFPair(sig2, gf2)  # same signature, should combine
    pair3 = SignatureGFPair(sig3, gf3)  # different signature

    # Initialize table and add
    table = SignatureTable()
    table.add(pair1)
    table.add(pair2)
    table.add(pair3)

    # Check the combined result for sig1
    combined = table.mapping[sig1]
    assert combined.terms == {1: 2, 2: 3}, f"Unexpected terms: {combined.terms}"

    # Check sig3 stored correctly
    assert table.mapping[sig3].terms == {0: 5}, "sig3 GF incorrect"

    # Optional: Print table
    print("Signature Table contents:")
    print(table.__repr__(multiline = True))

    print("✅ All tests passed.")
# test_signature_table()

def test_signature_transitions():
    # Test 1: Blank transition (0 -> 0, blank operation)
    sig = Signature([1, 0, 0], touches_top=True)
    gf = GeneratingFunction({1: 1})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=1, modify_to=0)
    assert new_pair is not None
    assert new_pair.signature.states == [1, 0, 0]
    assert new_pair.generating_function.terms == {1: 1}

    # Test 2: Blank transition (2 -> 2, blank operation)
    sig = Signature([4, 3, 2])
    gf = GeneratingFunction({4: 7})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=1, modify_to=1)
    assert new_pair is not None
    assert new_pair.signature.states == [4, 3, 2]
    assert new_pair.generating_function.terms == {5: 7}

    # Test 3: Blank transition (0 -> 3, blank operation)
    sig = Signature([4, 0, 0, 2])
    gf = GeneratingFunction({6: 1})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=1, modify_to=1)
    assert new_pair is not None
    assert new_pair.signature.states == [4, 3, 0, 2]
    assert new_pair.generating_function.terms == {7: 1}

    # Test 4.1: Hat operation (upwards)
    sig = Signature([4, 0, 4, 3, 0, 2, 2], touches_top=True, touches_bottom=True)
    gf = GeneratingFunction({1: 1})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=6, modify_to=1)
    assert new_pair is not None
    assert new_pair.signature.states == [4, 0, 3, 3, 0, 3, 2]
    assert new_pair.generating_function.terms == {2: 1}  # multiplied by x

    # Test 4.2: Hat operation (downwards)
    sig = Signature([ 4, 3, 4, 3, 2, 0, 2], touches_top=True, touches_bottom=True)
    gf = GeneratingFunction({1: 2})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=2, modify_to=1)
    assert new_pair is not None
    assert new_pair.signature.states == [ 4, 3, 3, 3, 3, 0, 2]
    assert new_pair.generating_function.terms == {2: 2}

    # Test 5.1 : Over-line (upwards and turn to isolated)
    sig = Signature([4, 0, 4, 3, 2, 0, 2, 0], touches_top=True, touches_bottom=True)
    gf = GeneratingFunction({0: 5})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=6, modify_to=0)
    assert new_pair is not None
    assert new_pair.signature.states == [1, 0, 4, 3, 2, 0, 0, 0]
    assert new_pair.generating_function.terms == {0: 5}

    # Test 5.2 : Over-line (upwards and change intermediate site)
    sig = Signature([4, 3, 3, 0, 4, 3, 2, 2, 0], touches_top=True, touches_bottom=True)
    gf = GeneratingFunction({0: 5})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=7, modify_to=0)
    assert new_pair is not None
    assert new_pair.signature.states == [4, 3, 2, 0, 4, 3, 2, 0, 0]
    assert new_pair.generating_function.terms == {0: 5}

    #  Test 5.3: Over-line (downward and turn to isolated)
    sig = Signature([0, 0, 4, 0, 0, 2, 0, 0], touches_top=True, touches_bottom=True)
    gf = GeneratingFunction({0: 3})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=2, modify_to=0)
    assert new_pair is not None
    assert new_pair.signature.states == [0, 0, 0, 0, 0, 1, 0, 0]
    assert new_pair.generating_function.terms == {0: 3}

    # Test 5.4: Over-line (downward and change intermediate site)
    sig = Signature([0, 0, 4, 3, 3, 2, 0, 0], touches_top=True, touches_bottom=True)
    gf = GeneratingFunction({0: 2})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=2, modify_to=0)
    assert new_pair is not None
    assert new_pair.signature.states == [0, 0, 0, 4, 3, 2, 0, 0]
    assert new_pair.generating_function.terms == {0: 2}

    # Test 5.5: Over-line (downward and change intermediate site and nesting)
    sig = Signature([4, 0, 4, 3, 3, 2, 0, 3, 3, 2], touches_top=True, touches_bottom=True)
    gf = GeneratingFunction({0: 2})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=0, modify_to=0)
    assert new_pair is not None
    assert new_pair.signature.states == [0, 0, 4, 3, 3, 2, 0, 4, 3, 2]
    assert new_pair.generating_function.terms == {0: 2}

    # Test 6: Impossible case
    sig = Signature([1, 1, 1], touches_top=True)
    gf = GeneratingFunction({0: 2})
    pair = SignatureGFPair(sig, gf)
    result = pair.transition(row=1, modify_to=0)
    assert result is None

    print("✅ All signature transition tests passed.")
# test_signature_transitions()

def test_can_add():
    # Test 1: Valid add condition – only row 2 is 1
    sig = Signature([0, 0, 1, 0, 0], touches_top=True, touches_bottom=True)
    assert sig.can_add(2) == True, "Test 1 failed: should allow add at row 2"

    # Test 2: Invalid – multiple 1s
    sig = Signature([1, 0, 1, 0, 0], touches_top=True, touches_bottom=True)
    assert sig.can_add(2) == False, "Test 2 failed: should not allow add, multiple 1s"

    # Test 3: Invalid – 1 at wrong row
    sig = Signature([0, 0, 1, 0, 0], touches_top=True, touches_bottom=False)
    assert sig.can_add(1) == False, "Test 3 failed: should not allow add, 1 not at row 1"
    assert sig.can_add(2) == False, "Test 3 failed: should allow not add at row 2"

    print("✅ All can_add tests passed.")
# test_can_add()

def test_generating_function_min_exp_direct():
    # Test 1: Single term GF
    gf = GeneratingFunction({3: 5})
    assert gf.min_exp == 3, f"Expected min_exp = 3, got {gf.min_exp}"

    # Test 2: Multiple term GF
    gf = GeneratingFunction({5: 1, 2: 9, 9: 4})
    assert gf.min_exp == 2, f"Expected min_exp = 2, got {gf.min_exp}"
# test_generating_function_min_exp_direct()

def test_generating_function_min_exp_transition():
    # Test 3: Transition with multiply_by_x (occupied cell)
    sig = Signature([0, 1])
    gf = GeneratingFunction({1: 1})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=0, modify_to=1)  # should multiply GF by x
    assert new_pair is not None
    assert new_pair.generating_function.min_exp == 2, \
        f"Expected min_exp = 2, got {new_pair.generating_function.min_exp}"

    # Test 4: Transition with no x multiplication (empty cell)
    sig = Signature([4, 2])
    gf = GeneratingFunction({3: 1, 4: 3})
    pair = SignatureGFPair(sig, gf)
    new_pair = pair.transition(row=0, modify_to=0)
    assert new_pair is not None
    assert new_pair.generating_function.min_exp == 3, \
        f"Expected min_exp = 3, got {new_pair.generating_function.min_exp}"
# test_generating_function_min_exp_transition()


# Compare with OEIS


In [55]:
import requests

def fetch_fixed_polyomino_counts(n_max):
    """
    Fetch the fixed polyomino counts from OEIS (A001168) up to n_max.
    Returns a list counts[1..n_max].
    """
    url = "https://oeis.org/A001168/b001168.txt"
    resp = requests.get(url)
    resp.raise_for_status()

    counts = [None] * (n_max + 1)
    for line in resp.text.splitlines():
        if line.startswith("#"):
            continue
        parts = line.split()
        if not parts:
            continue
        i, val = int(parts[0]), int(parts[1])
        if i <= n_max:
            counts[i] = int(val)
    return counts

n=17 # 17 is executed in more or less 7 seconds right now on google colab.
poly_enum = PolyominoEnumerator(n)
gf = poly_enum.run()
counts = fetch_fixed_polyomino_counts(n)
# enumerate only coefficients whose exponents are lower than n
print(gf)
for i, coeff in enumerate(gf.terms.values()):
    if i > n:
        break
    if coeff == counts[i]:
        print(f"{i}: {coeff} COUNTED CORRECTLY")
    else:
        print(f"{i}: {coeff} COUNTED INCORRECTLY")

1x^0 + 1x^1 + 2x^2 + 6x^3 + 19x^4 + 63x^5 + 216x^6 + 760x^7 + 2725x^8 + 9910x^9 + 36446x^10 + 135268x^11 + 505861x^12 + 1903890x^13 + 7204874x^14 + 27394666x^15 + 104592937x^16 + 400795844x^17
0: 1 COUNTED CORRECTLY
1: 1 COUNTED CORRECTLY
2: 2 COUNTED CORRECTLY
3: 6 COUNTED CORRECTLY
4: 19 COUNTED CORRECTLY
5: 63 COUNTED CORRECTLY
6: 216 COUNTED CORRECTLY
7: 760 COUNTED CORRECTLY
8: 2725 COUNTED CORRECTLY
9: 9910 COUNTED CORRECTLY
10: 36446 COUNTED CORRECTLY
11: 135268 COUNTED CORRECTLY
12: 505861 COUNTED CORRECTLY
13: 1903890 COUNTED CORRECTLY
14: 7204874 COUNTED CORRECTLY
15: 27394666 COUNTED CORRECTLY
16: 104592937 COUNTED CORRECTLY
17: 400795844 COUNTED CORRECTLY
